# Работа с БД: MongoDB

MongoDB - это т.н. **документоориентированная БД**, иногда их ещё называют бессхемными (schema-less).

В противоположность реляционным БД, где есть таблицы со строками, MongoDB оперирует другими понятиями:
* database - база данных, как в Postgres
* collection - коллекция, аналог таблицы
* document - документ, аналог строки в таблице

В отличие от реляционных БД, где все строки в таблице имеют одинаковые поля, каждый документ в коллекции MongoDB потенциально может иметь свой уникальный набор полей.

Подключёние происходит с помощью библиотеку **pymongo**, для подключения к БД нужно авторизоваться. Попробуем подключиться к БД и вытащить список доступных коллекций.

In [3]:
from pymongo import MongoClient

mongo_connection = {
    "host": "dsstudents.skillbox.ru",
    "port": 27017,
    "user": "students",
    "password": "X63673t47Gl03Sq",
    "authSource": "movies"
}

mongo = MongoClient('mongodb://%s:%s@%s:%s/?authSource=%s' % (
    mongo_connection['user'], mongo_connection['password'],
    mongo_connection['host'], mongo_connection['port'], mongo_connection['authSource'])
)
db = mongo["movies"]

print("Коллекции, доступные в MongoDB: %s" % db.list_collection_names())

Коллекции, доступные в MongoDB: ['tags']


Можно посчитать самую простую статистику - количество документов в коллекции с помощью функции *.count()*

In [4]:
collection = db['tags']
print("Число документов в коллекции %s" % collection.estimated_document_count())

Число документов в коллекции 158680


In [5]:
mongo_cursor = collection.find().limit(5)
print("Результат выборки: объект типа cursor %s\n" % mongo_cursor)

# пройдёмся по курсору и посмотрим, что внутри
cursor_items = [item for item in mongo_cursor]
print("Сожержимое курсора:\n%s\n" % cursor_items)
print("Поля элемента курсора %s" % list(cursor_items[0].keys()))

Результат выборки: объект типа cursor <pymongo.cursor.Cursor object at 0x7fef3c25beb8>

Сожержимое курсора:
[{'_id': ObjectId('5c822402c0669da98bd5081e'), 'id': 931, 'name': 'jealousy'}, {'_id': ObjectId('5c822402c0669da98bd5081f'), 'id': 4290, 'name': 'toy'}, {'_id': ObjectId('5c822402c0669da98bd50820'), 'id': 5202, 'name': 'boy'}, {'_id': ObjectId('5c822402c0669da98bd50821'), 'id': 6054, 'name': 'friendship'}, {'_id': ObjectId('5c822402c0669da98bd50822'), 'id': 9713, 'name': 'friends'}]

Поля элемента курсора ['_id', 'id', 'name']


В метод *.find()* можно подставить т.н. селектор - это словарь, который помогает оставить в выдачё только нужные поля. Селектор задаётся атрибутом *projection*

In [6]:
selector = {'name': True}
mongo_cursor = collection.find(projection=selector).limit(5)
cursor_items = [item for item in mongo_cursor]
print("Содержимое курсора (оставляем только поле 'name'):\n%s\n" % cursor_items)

# Нам мешается поле "_id" - можем выключить его
selector = {'_id': False}
mongo_cursor = collection.find(projection=selector).limit(5)
cursor_items = [item for item in mongo_cursor]
print("Сожержимое курсора (выключаем _id):\n%s\n" % cursor_items)

#  можем выключить  поле "_id" и включить "name"
selector = {'_id': False, 'name': True}
mongo_cursor = collection.find(projection=selector).limit(5)
cursor_items = [item for item in mongo_cursor]
print("Сожержимое курсора без лишних полей:\n%s\n" % cursor_items)

Содержимое курсора (оставляем только поле 'name'):
[{'_id': ObjectId('5c822402c0669da98bd5081e'), 'name': 'jealousy'}, {'_id': ObjectId('5c822402c0669da98bd5081f'), 'name': 'toy'}, {'_id': ObjectId('5c822402c0669da98bd50820'), 'name': 'boy'}, {'_id': ObjectId('5c822402c0669da98bd50821'), 'name': 'friendship'}, {'_id': ObjectId('5c822402c0669da98bd50822'), 'name': 'friends'}]

Сожержимое курсора (выключаем _id):
[{'id': 931, 'name': 'jealousy'}, {'id': 4290, 'name': 'toy'}, {'id': 5202, 'name': 'boy'}, {'id': 6054, 'name': 'friendship'}, {'id': 9713, 'name': 'friends'}]

Сожержимое курсора без лишних полей:
[{'name': 'jealousy'}, {'name': 'toy'}, {'name': 'boy'}, {'name': 'friendship'}, {'name': 'friends'}]



Фильтровать можно с помощью параметра *.filter()*

In [7]:
selector = {'name': 'toy'}
exclude_id = {'_id': False}
mongo_cursor = collection.find(projection=exclude_id, filter={'name': 'toy'}).limit(5)
cursor_items = [item for item in mongo_cursor]
print("Сожержимое курсора (оставляем только 'name'=='toy'):\n%s\n" % cursor_items)

Сожержимое курсора (оставляем только 'name'=='toy'):
[{'id': 4290, 'name': 'toy'}, {'id': 4290, 'name': 'toy'}, {'id': 4290, 'name': 'toy'}, {'id': 4290, 'name': 'toy'}, {'id': 4290, 'name': 'toy'}]



MongoDB позволяет также выполнять сложные агрегирующие запросы средствами СУБД

In [8]:
pipline = [
    {"$group":
        {"_id": "$name",
         "tag_count":
            {"$sum": 1}
         }
     },
    {"$sort":
        {"tag_count": -1}
     },
    {"$limit": 5}
]

print([i for i in collection.aggregate(pipline)])

[{'_id': 'woman director', 'tag_count': 3115}, {'_id': 'independent film', 'tag_count': 1930}, {'_id': 'murder', 'tag_count': 1308}, {'_id': 'based on novel', 'tag_count': 835}, {'_id': 'musical', 'tag_count': 734}]
